In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [40]:
data = pd.read_csv('order_brush_order.csv')
data.head()

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55


In [41]:
len(data)

222750

In [42]:
#no duplicated order id
data[data['orderid'].duplicated()]

,orderid,shopid,userid,event_time


In [43]:
len(data['userid'].unique())

93883

In [44]:
#number of shops
len(data['shopid'].unique())

18770

In [45]:
#convert the string to datetime
data['event_time'] = pd.to_datetime(data['event_time'])

In [46]:
def calculate_concentration(n_orders_one_hour, n_uniq_buyers_one_hour):
    return n_orders_one_hour / n_uniq_buyers_one_hour


In [47]:
#result dataframe
results = pd.DataFrame(data=data['shopid'].unique(), columns=['shopid'])
results['userid'] = None

results.head()

,shopid,userid
0,93950878,None
1,156423439,None
2,173699291,None
3,63674025,None
4,127249066,None


In [48]:
uniq_shop_id = data['shopid'].unique()
for i in range(len(uniq_shop_id)):
    s_id = uniq_shop_id[i]
    df = data[data['shopid'] == s_id].sort_values(by='event_time')
    
    temp = {}
    
    #get all the event_time for the shop
    event_time = df['event_time'].tolist()
    
    #for every event_time, get the records within next 1 hour
    for t in event_time:
        start = t
        end = t + timedelta(hours=1)
        
        records = df[(df['event_time'] >= start) & (df['event_time'] <= end)]
        #print(records)
        concentration = calculate_concentration(len(records['orderid']), len(records['userid'].unique()))
        
        if concentration >= 3:
            print(records)
            suspicious_user = pd.DataFrame(records.groupby(by='userid').count().orderid).reset_index().rename(columns={'userid':'userid','orderid':'count'}).set_index('userid').to_dict().values()
            dic = list(suspicious_user)[0]
            
            for k, v in dic.items():
                if k in temp:
                    temp[k] += v
                else:
                    temp[k] = v
            print(dic)
        
    highest_id = [k for k, v in temp.items() if v == max(temp.values())]
    highest_id = sorted(highest_id)
    #print(highest_id)
    results.loc[results['shopid']==s_id,'userid'] = '&'.join([str(x) for x in highest_id])

               orderid   shopid     userid          event_time
21027   31243838739795  1175477  122277324 2019-12-28 22:50:39
9409    31244142101712  1175477  122277324 2019-12-28 22:55:42
190109  31244747422135  1175477  122277324 2019-12-28 23:05:47
{122277324: 3}
               orderid    shopid     userid          event_time
186125  31247365651612  66861410  213502289 2019-12-28 23:49:26
45551   31248087439155  66861410  213502289 2019-12-29 00:01:27
180814  31249068920318  66861410  213502289 2019-12-29 00:17:48
{213502289: 3}
               orderid   shopid   userid          event_time
119754  31158495379642  8715449  9753706 2019-12-27 23:08:15
148038  31158633469991  8715449  9753706 2019-12-27 23:10:33
111247  31158746900290  8715449  9753706 2019-12-27 23:12:26
{9753706: 3}
               orderid    shopid    userid          event_time
77275   31303367297188  58543771  61893096 2019-12-29 15:22:48
20524   31303675289511  58543771  61893096 2019-12-29 15:27:56
148343  31303724

              orderid    shopid     userid          event_time
82128  31498301819557  50713918  172106152 2019-12-31 21:31:42
78509  31498371248707  50713918  172106152 2019-12-31 21:32:52
82054  31498450405184  50713918  172106152 2019-12-31 21:34:10
{172106152: 3}
               orderid     shopid    userid          event_time
137772  31384786235011  169902791  50198835 2019-12-30 13:59:47
164364  31384939445555  169902791  50198835 2019-12-30 14:02:20
164908  31385012642562  169902791  50198835 2019-12-30 14:03:32
{50198835: 3}
               orderid  shopid     userid          event_time
23154   31460011525926   42818  170385453 2019-12-31 10:53:31
208306  31460217564275   42818  170385453 2019-12-31 10:56:57
107340  31460245896451   42818  170385453 2019-12-31 10:57:26
61085   31460381477302   42818  170385453 2019-12-31 10:59:41
74050   31460637497379   42818  170385453 2019-12-31 11:03:57
{170385453: 5}
               orderid  shopid     userid          event_time
208306  314602

               orderid     shopid    userid          event_time
219068  31276822905006  134968430  59725263 2019-12-29 08:00:22
158626  31277294092868  134968430  59725263 2019-12-29 08:08:14
211785  31278734731364  134968430  59725263 2019-12-29 08:32:14
{59725263: 3}
               orderid     shopid     userid          event_time
149045  31233827237697  171407673  211296094 2019-12-28 20:03:47
169720  31236482407035  171407673  211296094 2019-12-28 20:48:02
28742   31237199155096  171407673  211296094 2019-12-28 21:00:00
{211296094: 3}
               orderid    shopid     userid          event_time
58549   31117756947276  16001939  205729485 2019-12-27 11:49:16
182687  31118067895323  16001939  205729485 2019-12-27 11:54:28
96254   31119033888486  16001939  205729485 2019-12-27 12:10:34
{205729485: 3}
               orderid    shopid     userid          event_time
121457  31393625623459  16001939    1024838 2019-12-30 16:27:05
32417   31394222215336  16001939  205729485 2019-12-30 1

               orderid     shopid     userid          event_time
98144   31494423041272  150526730  108214177 2019-12-31 20:27:03
1199    31494631546707  150526730  108214177 2019-12-31 20:30:31
112204  31494916275956  150526730  108214177 2019-12-31 20:35:17
{108214177: 3}
              orderid    shopid     userid          event_time
42406  31400543198121  76668593  215224309 2019-12-30 18:22:23
70334  31402663042852  76668593  215224309 2019-12-30 18:57:43
93547  31402917150618  76668593  215224309 2019-12-30 19:01:57
{215224309: 3}
               orderid   shopid     userid          event_time
185045  31471594311732  4888564  143847348 2019-12-31 14:06:34
86088   31472846547416  4888564  143847348 2019-12-31 14:27:26
102528  31473047891382  4888564  143847348 2019-12-31 14:30:48
{143847348: 3}
               orderid     shopid     userid          event_time
141555  31148708357289  110641390  157946285 2019-12-27 20:25:08
209242  31148773701850  110641390  157946285 2019-12-27 20:26

               orderid    shopid    userid          event_time
210395  31262915739040  18190345  89014205 2019-12-29 04:08:35
70452   31263006003564  18190345  89014205 2019-12-29 04:10:06
68246   31263113086396  18190345  89014205 2019-12-29 04:11:54
{89014205: 3}
               orderid    shopid     userid          event_time
219797  31420231463935  51487211  214808165 2019-12-30 23:50:31
59594   31420578990887  51487211  214808165 2019-12-30 23:56:18
15156   31420876664321  51487211  214808165 2019-12-31 00:01:17
{214808165: 3}
               orderid     shopid     userid          event_time
12841   31472743407793  168388504  148215831 2019-12-31 14:25:43
214399  31472894074283  168388504  148215831 2019-12-31 14:28:14
133695  31473497472129  168388504  148215831 2019-12-31 14:38:18
{148215831: 3}
               orderid     shopid    userid          event_time
7981    31201171078952  140626153  86570212 2019-12-28 10:59:31
164098  31201303821342  140626153  86570212 2019-12-28 11:01

...

               orderid  shopid     userid          event_time
14364   31322202668438  195531  214992524 2019-12-29 20:36:42
97442   31322258450835  195531  214992524 2019-12-29 20:37:38
144961  31322312856508  195531  214992524 2019-12-29 20:38:33
118067  31322358985666  195531  214992524 2019-12-29 20:39:18
{214992524: 4}
               orderid    shopid     userid          event_time
57325   31227940378784  48970010  207880909 2019-12-28 18:25:41
13035   31228701368226  48970010  207880909 2019-12-28 18:38:21
129857  31229451930255  48970010  207880909 2019-12-28 18:50:51
{207880909: 3}
               orderid  shopid  userid          event_time
30685   31115953007150   10536  672345 2019-12-27 11:19:13
76124   31116816860970   10536  672345 2019-12-27 11:33:37
117649  31117448503731   10536  672345 2019-12-27 11:44:08
{672345: 3}
               orderid  shopid  userid          event_time
195783  31138151853986   10536  672345 2019-12-27 17:29:12
39004   31139160444394   10536  672345

               orderid  shopid     userid          event_time
67995   31230508726180  605561  181682008 2019-12-28 19:08:28
114320  31230618877066  605561  181682008 2019-12-28 19:10:19
49349   31230660361181  605561  181682008 2019-12-28 19:11:01
{181682008: 3}
               orderid    shopid    userid          event_time
121922  31307339930031  80890996  77374158 2019-12-29 16:28:59
47996   31307513704430  80890996  77374158 2019-12-29 16:31:53
174038  31307642756514  80890996  77374158 2019-12-29 16:34:03
{77374158: 3}
               orderid  shopid  userid          event_time
198614  31213473016957   10402   77819 2019-12-28 14:24:33
185351  31213764438384   10402   77819 2019-12-28 14:29:25
95249   31213825677675   10402   77819 2019-12-28 14:30:26
{77819: 3}
               orderid    shopid     userid          event_time
88968   31136056845868  87657406  214094512 2019-12-27 16:54:17
213690  31136315949109  87657406  214094512 2019-12-27 16:58:35
173829  31136640485660  87657406

               orderid     shopid     userid          event_time
99101   31419936681511  173318671  215301243 2019-12-30 23:45:36
145176  31420317186222  173318671  215301243 2019-12-30 23:51:57
118897  31420701726322  173318671  215301243 2019-12-30 23:58:22
{215301243: 3}
               orderid     shopid     userid          event_time
77165   31126810724007  153335627  111390406 2019-12-27 14:20:10
211141  31127043526649  153335627  111390406 2019-12-27 14:24:03
84451   31127405720634  153335627  111390406 2019-12-27 14:30:05
{111390406: 3}
               orderid    shopid     userid          event_time
61807   31337903237944  83366265  118860287 2019-12-30 00:58:24
218155  31337933469609  83366265  118860287 2019-12-30 00:58:53
94633   31337956447064  83366265  118860287 2019-12-30 00:59:16
168404  31337981857342  83366265  118860287 2019-12-30 00:59:42
145343  31338012619997  83366265  118860287 2019-12-30 01:00:12
159266  31338037453608  83366265  118860287 2019-12-30 01:00:37
{1

               orderid     shopid     userid          event_time
139013  31078990536510  192608876  213646699 2019-12-27 01:03:10
118431  31080726584116  192608876  213646699 2019-12-27 01:32:07
125161  31081343380651  192608876  213646699 2019-12-27 01:42:23
{213646699: 3}
               orderid    shopid    userid          event_time
29032   31153042776991  60587222  33794624 2019-12-27 21:37:22
215161  31153618140768  60587222  33794624 2019-12-27 21:46:59
72280   31153943582299  60587222  33794624 2019-12-27 21:52:23
{33794624: 3}
               orderid   shopid    userid          event_time
180363  31373590289622  3701311  80690628 2019-12-30 10:53:10
42456   31374066115559  3701311  80690628 2019-12-30 11:01:06
55359   31374248228283  3701311  80690628 2019-12-30 11:04:08
{80690628: 3}
               orderid    shopid     userid          event_time
117091  31492390912282  31614331  105848455 2019-12-31 19:53:10
76599   31492698117685  31614331  105848455 2019-12-31 19:58:18
29252

               orderid     shopid    userid          event_time
77622   31388723612210  122238789  51759862 2019-12-30 15:05:24
171556  31390417288058  122238789  51759862 2019-12-30 15:33:38
57564   31391879735211  122238789  51759862 2019-12-30 15:57:59
{51759862: 3}
               orderid     shopid     userid          event_time
221111  31297170398828  134418456  214778616 2019-12-29 13:39:31
173605  31297730334728  134418456  214778616 2019-12-29 13:48:51
58833   31299318112518  134418456  214778616 2019-12-29 14:15:18
{214778616: 3}
               orderid     shopid    userid          event_time
149874  31412792656549  152234237  36097212 2019-12-30 21:46:32
119274  31412948018117  152234237  36097212 2019-12-30 21:49:08
182868  31413489655606  152234237  36097212 2019-12-30 21:58:09
59611   31413692266261  152234237  36097212 2019-12-30 22:01:33
{36097212: 4}
               orderid     shopid    userid          event_time
119274  31412948018117  152234237  36097212 2019-12-30 21

In [49]:
results.loc[results['shopid']==s_id,'userid']

18769    
Name: userid, dtype: object

In [50]:
results['userid']

0         
1         
2         
3         
4         
5         
6         
7         
8         
9         
10        
11        
12        
13        
14        
15        
16        
17        
18        
19        
20        
21        
22        
23        
24        
25        
26        
27        
28        
29        
        ..
18740     
18741     
18742     
18743     
18744     
18745     
18746     
18747     
18748     
18749     
18750     
18751     
18752     
18753     
18754     
18755     
18756     
18757     
18758     
18759     
18760     
18761     
18762     
18763     
18764     
18765     
18766     
18767     
18768     
18769     
Name: userid, Length: 18770, dtype: object

In [35]:
results.to_csv('submission1.csv', header=True, index=False)

In [ ]:
results

In [ ]:
#testing

s_id = 16001939
df = data[data['shopid'] == s_id].sort_values(by='event_time')

temp = {}

#get all the event_time for the shop
event_time = df['event_time'].tolist()

#for every event_time, get the records within next 1 hour
for t in event_time:
    start = t
    end = t + timedelta(hours=1)

    records = df[(df['event_time'] >= start) & (df['event_time'] <= end)]

    concentration = calculate_concentration(len(records['orderid']), len(records['userid'].unique()))

    if concentration >= 3:
        suspicious_user = pd.DataFrame(records.groupby(by='userid').count().orderid).reset_index().rename(columns={'userid':'userid','orderid':'count'}).set_index('userid').to_dict().values()
        dic = list(suspicious_user)[0]
        
        for k, v in dic.items():
            if k in temp:
                temp[k] += v
            else:
                temp[k] = v

        
        
temp = {205729485: 88, 1024838: 88}
highest_id = [k for k, v in temp.items() if v == max(temp.values())]
highest_id = sorted(highest_id)
print(highest_id)

results.loc[results['shopid']==s_id,'userid'] = '&'.join([str(x) for x in highest_id])
print(results.loc[results['shopid']==s_id])

